# Parquet - Hvordan få oversikt og flate ut komplekse filstukturer

## Praktiske eksempler fra Sirius Næringsteamet

I Dapla er det filformatet Parquet som benyttes. 

Apache Parquet is a file format designed to support fast data processing for complex data, with several notable characteristics:
  
1. Columnar: Unlike row-based formats such as CSV or Avro, Apache Parquet is column-oriented – meaning the values of each table column are stored next to each other, rather than those of each record:
![Column](Parquet_column.png)
2. Open-source: Parquet is free to use and open source under the Apache Hadoop license, and is compatible with most Hadoop data processing frameworks.
3. Self-describing: In Parquet, metadata including schema and structure is embedded within each file, making it a self-describing file format. 

#### Hvordan gå fra dette:

![Column](parquet_raadata.PNG)

#### Til dette:

![Column](parquet_utflatet.PNG)

### Importerer nødvendige pakker - kode kjøres på Pyspark

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import *

#### Lister ut alle filer i en mappe på Dapla

In [ ]:
df1 = spark.read.path("/kilde/ske/skatt/naering/naeringsopplysninger/2020/syntetisk/*")
df1.show(10, False)

#### Lester inn ønsket Parquet-fil og tar ut en oversikt over filstrukturen

In [ ]:
df1 = spark.read.path("/kilde/ske/skatt/naering/naeringsopplysninger/2020/syntetisk/20210819")

df1.printSchema()

#### Tar ut første observasjon i datasettet

In [ ]:
df1.head(1)

#### Ønsker bare å se på tema "Balanse" for å få bedre oversikt¶

In [ ]:
df1 = spark.read.path("/kilde/ske/skatt/naering/naeringsopplysninger/2020/syntetisk/20210819").select('norskIdentifikator', 'balanse')
df1.printSchema()

#### Finnes 2 typer av datastukturer
    Struct
    Array

#### Behandling av STRUCT

*    sumAnleggsmiddelSkattemessigVerdi
*    sumAnleggsmiddelRegnskapsmessigVerdi
*
*omloepsmiddel
*    sumOmloepsmiddelSkattemessigVerdi
*    sumOmloepsmiddelRegnskapsmessigVerdi
*
*gjeldOgEgenkapital
*    sumLangsiktigGjeldSkattemessigVerdi
*    sumLangsiktigGjeldRegnskapsmessigVerdi
*    sumKortsiktigGjeldSkattemessigVerdi
*sumKortsiktigGjeldRegnskapsmessigVerdi
*    sumEgenkapital
*
*sumEiendelSkattemessigVerdi
*sumEiendelRegnskapsmessigVerdi
*sumGjeldOgEgenkapitalSkattemessigVerdi
*sumGjeldOgEgenkapitalRegnskapsmessigVerdi

### Eksempel på utflating av en enkelt STRUCT-variabel - sumAnleggsmiddelSkattemessigVerdi

In [ ]:
#Tar ut bare sumAnleggsmiddelSkattemessigVerdi og renamer beloep til sumAnleggsmiddelSkattemessigVerdi
balanse_struct = df1.select(df1.norskIdentifikator,\
                          df1.balanse.anleggsmiddel.sumAnleggsmiddelSkattemessigVerdi.beloep.alias('sumAnleggsmiddelSkattemessigVerdi'),\
                              )

balanse_struct.printSchema()

In [ ]:
#lister ut de 10 første observasjonene
balanse_struct.show(10, False)

### Eksempel på utflating av flere STRUCT-variabler i samme steg

In [ ]:
#tar ut flere variabler samtidig og renamer beloep-variabler
balanse_struct = df1.select(df1.norskIdentifikator,\
                          df1.balanse.anleggsmiddel.sumAnleggsmiddelSkattemessigVerdi.beloep.alias('sumAnleggsmiddelSkattemessigVerdi'),\
                          df1.balanse.anleggsmiddel.sumAnleggsmiddelRegnskapsmessigVerdi.beloep.alias('sumAnleggsmiddelRegnskapsmessigVerdi'),\
                          df1.balanse.omloepsmiddel.sumOmloepsmiddelSkattemessigVerdi.beloep.alias('sumOmloepsmiddelSkattemessigVerdi'),\
                          df1.balanse.omloepsmiddel.sumOmloepsmiddelRegnskapsmessigVerdi.beloep.alias('sumOmloepsmiddelRegnskapsmessigVerdi'),\
                                                                             )
balanse_struct.printSchema()
balanse_struct.show(10, False)

### Lagrer resultatet som en dataframe på dapla - som så kan leses inn i Python

In [ ]:
balanse_struct.write\
    .option("valuation", "INTERNAL")\
    .option("state", "INPUT")\
    .path('/produkt/skatt/naering/temp/parquet_demo/balanse_struct')

## Behandling av ARRAYS - balanseverdiForAnleggsmiddel

#### Arrays er en kompleks datastruktur som kan inneholde flere forekomster av en identifikator

In [ ]:
balanse_array = 'balanseverdiForAnleggsmidler'
balanse_array = df1.select('norskIdentifikator', 'balanse.anleggsmiddel.balanseverdiForAnleggsmidler.balanseverdiForAnleggsmiddel')

balanse_array.printSchema()
balanse_array.show(10, False)


#### Bruker EXPLODE funksjonen for å flate ut data

In [ ]:
balanse_array = 'balanseverdiForAnleggsmidler'
balanse_array = df1.select('norskIdentifikator', 'balanse.anleggsmiddel.balanseverdiForAnleggsmidler.balanseverdiForAnleggsmiddel')

balanse_array = balanse_array.select('norskIdentifikator', 'balanseverdiForAnleggsmiddel' )\
         .withColumn('sonavn', F.explode('balanseverdiForAnleggsmiddel')).select('norskIdentifikator', 'sonavn.*').drop('balanseverdiForAnleggsmiddel').na.fill(0)


balanse_array.show(10, False)
balanse_array.printSchema()

##### Nå er vi nesten i mål - Henter ut 'beloep'-verdiene og renamer disse

In [ ]:
balanse_array = balanse_array\
        .select('norskIdentifikator', 'id', 'anleggsmiddeltype', 'skattemessigVerdi.beloep', 'regnskapsmessigVerdi.beloep', 'overfoeresIkkeTilSkattemeldingen')\
        .toDF('norskIdentifikator', 'id', 'anleggsmiddeltype', 'skattemessigVerdi', 'regnskapsmessigVerdi', 'overfoeresIkkeTilSkattemeldingen')
balanse_array.show(10, False)
balanse_array.printSchema()

#### Lagrer resultatet som en dataframe på dapla - som så kan leses inn i Python


In [ ]:
balanse_array.write\
    .option("valuation", "INTERNAL")\
    .option("state", "INPUT")\
    .path('/produkt/skatt/naering/temp/parquet_demo/balanse_array')